קודים של הקדמה

In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
import fiona

In [2]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [3]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [4]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [5]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

לייצר טבלה עם כמות תלמידים לפי רשות

In [6]:
cwd = os.getcwd()

create_forecast_basic_folder_path = os.path.dirname(cwd)

sys.path.append(create_forecast_basic_folder_path)

In [7]:
muni_JTMT=up_load_shp(r'{}\background_files\muni_under_JTMT_ITM.shp'.format(cwd))


In [8]:
col=['CR_PNIM','Muni_Heb']
muni_JTMT=muni_JTMT[col]
muni_JTMT['CR_PNIM']=muni_JTMT['CR_PNIM'].astype(int)

In [9]:
# Define the directory path
directory_path = r'{}\background_files'.format(cwd)

# # Load each DataFrame separately
df1 = up_load_df(directory_path, 'cbs_student_2020_by_muni_3')
df2 = up_load_df(directory_path, 'cbs_student_2020_by_muni_1')
df3 = up_load_df(directory_path, 'cbs_student_2020_by_muni_2')

# # Concatenate the DataFrames
student_gov_by_muni = pd.concat([df1, df2, df3])

In [10]:
col=[ 'סמל_יישוב',
 'תלמידים_בבתי_ספר_יסודיים_תש_ף_2019_20',
 'תלמידים_בחטיבות_ביניים_תש_ף_2019_20',
 'תלמידים_בבתי_ספר_תיכוניים_תש_ף_2019_20']

student_gov_by_muni=student_gov_by_muni[col]

In [11]:
student_gov_by_muni=student_gov_by_muni.merge(muni_JTMT,left_on='סמל_יישוב',right_on='CR_PNIM')
student_gov_by_muni=student_gov_by_muni.drop_duplicates(subset='CR_PNIM',keep='first')


In [12]:
col=[ 'תלמידים_בבתי_ספר_יסודיים_תש_ף_2019_20',
 'תלמידים_בחטיבות_ביניים_תש_ף_2019_20',
 'תלמידים_בבתי_ספר_תיכוניים_תש_ף_2019_20']

In [13]:
student_gov_by_muni['student_gov']=student_gov_by_muni[col].sum(axis=1)

In [14]:
student_gov_by_muni.to_excel('gov_muni_students.xlsx')